In [5]:
# Задание:
# Для каждой услуги из нового прайса клиники: service_name (main feature)
# Предложить топ-5 наиболее подходящих эталонных услуг: local_name (target)

# Расшифровка признаков: 
# local_id - Уникальный идентификатор услуги в эталонном прайсе;
# type - Категория или тип услуги, например, лабораторные тесты;
# gt_type_name - Подкатегория услуги, например, Биохимия;
# parent_id_name - Родительская категория услуги, например, Лабораторная диагностика;
# local_name - Эталонное название услуги;
# lpu_name - Название клиники;
# service_name - Название услуги в клинике;


# Данные из эталонного прайса
# local_data = {
#     "local_id": 5927.0,
#     "type": "laboratory_tests",  # Категория услуги
#     "gt_type_name": "Биохимия",  # Подкатегория услуги
#     "parent_id_name": "Лабораторная диагностика",  # Родительская категория
#     "local_name": "Varicella Zoster Virus, IgG"  # Эталонное название услуги
# }

# # Данные клиники
# clinic_data = {
#     "lpu_name": "Инвитро",  # Название клиники
#     "service_name": "Антитела класса IgG к вирусу Varicella-Zoster"  # Название услуги в клинике
# }

### Ожидаемые результаты:
# 1. **Код решения**:
#     - Код должен включать:
#         - EDA данных
#         - Алгоритм для поиска топ-5 эталонных услуг для каждой услуги из прайса клиники
#         - Расчёт метрик качества (Precision@k, Recall@k, F-score, MRR, NDCG etc)
#         - Решение необходимо отправить в гит
# 2. **Описание подхода**:
#     - Краткое описание используемой модели и методов, которые были применены для обучения модели и сопоставления услуг
#     - Обоснование выбора метрик и их использования
# 3. **Оценка результатов**:
#     - Привести примеры результатов для нескольких услуг из нового прайса клиники (например, "Антитела класса IgG к вирусу Varicella-Zoster")
#     - Проанализировать результаты и предложить возможные улучшения

# ### Что оценивается:
# 1. **Техническая реализация**: Качество реализации модели и алгоритма расчёта метрик
# 2. **Использование современных методов**: Применение современных языковых моделей и подходов
# 3. **Оценка результатов**: Умение интерпретировать метрики и анализировать результаты работы модели
# 4. Чистота кода и его воспроизводимость



import numpy as np
from tqdm import tqdm
import pandas as pd
from langchain_community.embeddings import HuggingFaceEmbeddings
import pickle
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
class RankSystem():
    def __init__(self, 
                 path_to_file = './merged_df.csv',
                 loaded_llm = True,
                 ) -> None:
        
        self.path_to_file = path_to_file
        self.data = pd.read_csv(self.path_to_file, sep=',', encoding='utf-8')
        self.loaded_llm = loaded_llm
        print(f'Исходный датасет:\n{self.data.head(3)}\n')

        if self.loaded_llm == False:
            self.bge_m3 = HuggingFaceEmbeddings(model_name = './models/embedding/')

        # прочие атрибуты класса
        self.embedding_data_dict = None
        self.service_features = None
        self.target_features = None
        self.score_matrix = None


    def eda(self,):
        
        print(f'общая информация по dataset')
        self.data.info()
        print('\n')
        # проверим какие столбцы содержат пропуски
        print(f'столбцы со значением None:\n{self.data.isna().any(axis=0)}\n')

        # строки с пропусками
        print(f'строки со значением None:\n{self.data[self.data.isna().any(axis=1)]}\n')

        # переведем local_id эталонных услуг к int
        self.data['local_id'] = self.data['local_id'].astype(int)

        print("""В задаче важную роль играют две колонки: local_name и service_name.
        Другие признаки в целом отражают дополнительную информацию и могут быть использованы в описании рекомендованных услуг.
        Пропущеннные значения в local_name и service_name отсутствуют\n""")
        
        # отберем необходимые столбцы
        columns_names = list(self.data.keys())
        print(f'columns_names:\n{columns_names}\n')

        # local_id относится к local_name в эталонном прайсе
        clean_data = self.data[['local_id', 'local_name', 'service_name', 'lpu_name']].copy()
        clean_data.isna().any()

        # число уникальных услуг
        local_name = clean_data['local_name'].unique()
        service_name = clean_data['service_name'].unique()
        print(
            f"""Число уникальных local_name услуг:{clean_data['local_name'].unique().__len__()}, 
    Число уникальных service_name услуги: {clean_data['service_name'].unique().__len__()}"""
        )

        # найдем эмбеддинги local_name, service_name  
        if self.loaded_llm == False:
            embedding_dict = {
            'local_name': self.build_embedding(local_name),
            'service_name': self.build_embedding(service_name)
            }

            with open('./embedding_dict.pickle', 'wb') as file:
                pickle.dump(embedding_dict, file)
            with open('./embedding_dict.pickle', 'rb') as file:
                self.embedding_data_dict = pickle.load(file)
        else:
            with open('./embedding_dict.pickle', 'rb') as file:
                self.embedding_data_dict = pickle.load(file)
        # print(f'embedding_data_dict: \n{self.embedding_data_dict}')

    def build_embedding(self, data):
        embeddings = {}
        for service in tqdm(data, mininterval = 0.5, desc='Total', total=data.__len__()):
            # нормализация текста
            service = service.lower().strip()
            # построение вектора
            embeddings[service] = self.bge_m3.embed_query(service)
        return embeddings
    
    def build_dataset(self,):

        # feature_i -> [target_0, ..., target_4]
        # разделяем услуги клиники от эталонных услуг из прейскуранта
        # каждая услуга есть пара (описание услуги, embedding)
        self.service_features = [(k,v) for k,v in self.embedding_data_dict['service_name'].items()]
        self.target_features = [(k,v) for k,v in self.embedding_data_dict['local_name'].items()]
        del self.embedding_data_dict

        # формирование матрицы признаков и таргетов
        features_array = np.array([service[1] for service in self.service_features])
        target_array = np.array([service[1] for service in self.target_features])

        # score_matrix[i,j] -> similarity score для i-услуги клиники и j-эталонной услуги из прайса
        self.score_matrix = cosine_similarity(features_array, target_array)
        print(f'размерность матрицы схожести: {self.score_matrix.shape}\n')
        return self.service_features, self.target_features

    def make_prediction(self, 
                        TOP_K,
                        service_name_idx,
                        ):
        service_title = self.service_features[service_name_idx][0]
        # print(f'название услуги: {service_title}\n')

        ranked_services_list = sorted(
            [(idx, score) for idx, score in enumerate(self.score_matrix[service_name_idx])],
            reverse=True,
            key=lambda x: x[1]
        )[:TOP_K]

        ranked_services = []
        for itm in ranked_services_list:
            ranked_services.append((self.target_features[itm[0]][0], f'score: {round(itm[1],3)}'))

        print(f'Услуга из прайса клиники: {service_title}\nCписок эталонных услуг: {ranked_services}')

In [7]:
model = RankSystem(
    path_to_file = './merged_df.csv',
    loaded_llm = True
)

Исходный датасет:
                                          local_name  local_id  parent_id  \
0                     Выявление стресса и его стадии    7626.0       64.0   
1  Клинический анализ крови с микроскопией лейкоц...    5523.0       64.0   
2  Лейкоцитарная формула (с обязательной микроско...    5524.0       64.0   

               type     gt_type_name            parent_id_name site_active  \
0  laboratory_tests         Комплекс  Лабораторная диагностика        True   
1  laboratory_tests  Общеклинические  Лабораторная диагностика        True   
2  laboratory_tests         Биохимия  Лабораторная диагностика        True   

                                        service_name lpu_name  
0                  Комплексное исследование «Стресс»  Инвитро  
1  Клинический анализ крови: общий анализ, лейкоф...  Инвитро  
2  Лейкоцитарная формула (дифференцированный подс...  Инвитро  



In [8]:
model.eda()

общая информация по dataset
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23700 entries, 0 to 23699
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   local_name      23700 non-null  object 
 1   local_id        23700 non-null  float64
 2   parent_id       20876 non-null  float64
 3   type            20876 non-null  object 
 4   gt_type_name    20876 non-null  object 
 5   parent_id_name  20876 non-null  object 
 6   site_active     20876 non-null  object 
 7   service_name    23700 non-null  object 
 8   lpu_name        23700 non-null  object 
dtypes: float64(2), object(7)
memory usage: 1.6+ MB


столбцы со значением None:
local_name        False
local_id          False
parent_id          True
type               True
gt_type_name       True
parent_id_name     True
site_active        True
service_name      False
lpu_name          False
dtype: bool

строки со значением None:
                                 

In [9]:
service_features, target_features = model.build_dataset()

размерность матрицы схожести: (14390, 4047)



In [10]:
# покажем название услуги в клинике 
TOP_K = 5
desc_service = [i[0] for i in service_features]
example = [
    'd-димер',
    'комплексное исследование «стресс»',
    'холестерин общий (холестерин, cholesterol total)'
]


for context in example:
    service_name_idx = desc_service.index(context)
    model.make_prediction(
        TOP_K,
        service_name_idx
    )

Услуга из прайса клиники: d-димер
Cписок эталонных услуг: [('d-димер', 'score: 1.0'), ('тест ширмера (двухсторонний)', 'score: 0.567'), ('дигоксин', 'score: 0.533'), ('асимметричный диметиларгинин', 'score: 0.531'), ('дифенин', 'score: 0.52')]
Услуга из прайса клиники: комплексное исследование «стресс»
Cписок эталонных услуг: [('комплексное исследование, осм', 'score: 0.717'), ('комплексная оценка оксидативного стресса (7 параметров)', 'score: 0.649'), ('выявление стресса и его стадии', 'score: 0.638'), ('флороценоз - комплексное исследование', 'score: 0.632'), ('комплексное исследование на гормоны (6 показателей)', 'score: 0.611')]
Услуга из прайса клиники: холестерин общий (холестерин, cholesterol total)
Cписок эталонных услуг: [('холестерол общий', 'score: 0.77'), ('холестерол – липопротеины высокой плотности (лпвп)', 'score: 0.631'), ('холестерол - не-лпвп (холестерол, не связанный с липопротеинами высокой плотности)', 'score: 0.622'), ('простатспецифический антиген общий (пса общи